In [52]:
f = open("TheDaVinciCode.txt", "r", encoding="utf-8")

book_text = f.read()

f.close()

In [53]:
import torch
from transformers import BertTokenizer, BertForTokenClassification

In [54]:
# Load a pre-trained BERT model that has been fine-tuned for NER
model = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels=18)
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [55]:
book_text = book_text.replace("\n", " ")
book_text = book_text.replace("  ", " ")
book_text = book_text[:5000]

In [65]:
# set maximum sequence length and BERT model
max_seq_length = 512
model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-cased')

# tokenize book text and split into smaller sequences

# Tokenize the input text using the BERT tokenizer
tokens = tokenizer.tokenize(book_text)

# Add the special [CLS] and [SEP] tokens to mark the beginning and end of the input text
tokens = ["[CLS]"] + tokens + ["[SEP]"]
token_ids = tokenizer.convert_tokens_to_ids(tokens)
segments = [0] * len(token_ids)
input_ids = []
attention_masks = []
while len(token_ids) > 0:
    if len(token_ids) > max_seq_length - 2:
        input_id = [tokenizer.cls_token_id] + token_ids[:max_seq_length-2] + [tokenizer.sep_token_id]
        attention_mask = [1] * len(input_id)
        token_ids = token_ids[max_seq_length-2:]
    else:
        input_id = [tokenizer.cls_token_id] + token_ids + [tokenizer.sep_token_id]
        attention_mask = [1] * len(input_id)
        padding = [0] * (max_seq_length - len(input_id))
        input_id += padding
        attention_mask += padding
        token_ids = []
    input_ids.append(input_id)
    attention_masks.append(attention_mask)

# convert input sequences to tensors and feed into model
outputs = []
for i in range(len(input_ids)):
    input_ids_tensor = torch.tensor([input_ids[i]])
    attention_mask_tensor = torch.tensor([attention_masks[i]])
    with torch.no_grad():
        output = model(input_ids_tensor, attention_mask=attention_mask_tensor)
    outputs.append(output[0])

# concatenate outputs and extract relevant information
# output_tensor = torch.cat(outputs, dim=1)
# output_tokens = tokenizer.convert_ids_to_tokens(output_tensor[0])
output_tensor = model(input_ids, attention_mask=attention_mask)
output_tokens = tokenizer.convert_ids_to_tokens(output_tensor[0].tolist())

# find all instances of "Robert Langdon"
robert_langdon_indices = []
print(output_tensor)
for i, token in enumerate(output_tokens):
    # if token == "robert" and output_tokens[i+1] == "langdon":
    if token == "robert":
        robert_langdon_indices.append(i)

# print the sentence containing each instance of "Robert Langdon"
for index in robert_langdon_indices:
    start = max(0, index - 20)
    end = min(len(output_tokens), index + 20)
    sentence = model.tokenizer.convert_tokens_to_string(output_tokens[start:end])
    print(sentence)

Using cache found in /Users/kristianvankuijk/.cache/torch/hub/huggingface_pytorch-transformers_main
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


AttributeError: 'list' object has no attribute 'size'

In [49]:
robert_langdon_indices

[]